# BirdNet sandbox code
This is a small collection of code to manipulate and analyse recording of bird song using the BirdNet Analyzer model to classify bird species based on recordings. The code is intended for use using Google Colab but can be easily altered for use on a local machine.  

In [ ]:
!git clone https://github.com/weharris/birdnetTest

In [ ]:
cd birdnetTest/

In [ ]:
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

In [ ]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

## Code for multiple recordings

In [5]:
# get the data
import os

dataDir = 'data/EdH20240427'
file_names = os.listdir(dataDir)
# Get the file names
print(file_names)

['BAT01_20240427_050000.wav', 'BAT01_20240427_060000.wav', 'BAT01_20240427_070000.wav', 'BAT01_20240427_080000.wav', 'BAT01_20240427_100000.wav', 'BAT01_20240427_110000.wav', 'BAT01_20240427_120000.wav', 'BAT01_20240427_130000.wav', 'BAT01_20240427_140000.wav', 'BAT01_20240427_150000.wav']


In [ ]:
# This cell uses BirdNET-Analyzer to analyze multiple recordings.
# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=20

# Create a dictionary to store the results
results = {}

# Loop through all file names
for file_name in file_names:
    # Include the correct path to the data files
    file_path = os.path.join(dataDir, file_name)
    recording = Recording(
        analyzer,
        file_path,  # path to the recording file
        lat=testamp[0], # latitude of the recording location
        lon=testamp[1], # longitude of the recording location
        date=datetime(year=2024, month=4, day=20), # use date or week_48
        min_conf=0.25,
    )
    # Analyze the recording and store the result in the dictionary
    recording.analyze()
    results[file_name] = recording.detections

In [ ]:
# for Multiple Files and list of spp and counts
import pandas as pd

# Convert the nested dictionaries into a list of dictionaries
data = []
for file_name, detections in results.items():
    for detection in detections:
        detection['file_name'] = file_name  # Add the file name to the detection dictionary
        data.append(detection)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Group by file_name and common_name, and count the number of detections for each
counts = df.groupby(['common_name','file_name' ]).size().unstack(fill_value=0)

# The result is a DataFrame where each row corresponds to a file, each column corresponds to a bird species,
# and the cell values are the counts of detections for each species in each file

print(counts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Create a dictionary that maps the old labels to the new labels
new_labels = {'BAT01_20240427_050000.wav': '05:00',
              'BAT01_20240427_060000.wav': '06:00',
              'BAT01_20240427_070000.wav': '07:00',
              'BAT01_20240427_080000.wav': '08:00',
              'BAT01_20240427_100000.wav': '10:00',
              'BAT01_20240427_110000.wav': '11:00',
              'BAT01_20240427_120000.wav': '12:00',
              'BAT01_20240427_130000.wav': '13:00',
              'BAT01_20240427_140000.wav': '14:00',
              'BAT01_20240427_150000.wav': '15:00'}

# Replace the column names of the DataFrame
counts.rename(columns=new_labels, inplace=True)

# Sum the counts for each species and sort in descending order
sorted_species = counts.sum(axis=1).sort_values(ascending=False).index

# Reorder the rows of the DataFrame
counts = counts.loc[sorted_species]

# Create a larger figure to accommodate the plot
plt.figure(figsize=(10, len(counts.index) * 0.2))  # Adjust the size as needed

# Create a colormap that uses gray for zero detections
cmap = colors.ListedColormap(sns.color_palette("YlGnBu", 256))
cmap.set_under((0.85, 0.85, 0.85))  # Light gray
#cmap.set_under("gray")

# Create the heatmap
sns.heatmap(counts, cmap=cmap, vmin=0.01)  # Set vmin to a small value so that zero detections are colored gray

plt.title('Bird species detections per hour')
plt.ylabel('Species')
plt.xlabel('Time')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
# Move the x-axis labels to the top
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate the number of detections for each file
new_labels = {'BAT01_20240427_050000.wav': '05:00',
              'BAT01_20240427_060000.wav': '06:00',
              'BAT01_20240427_070000.wav': '07:00',
              'BAT01_20240427_080000.wav': '08:00',
              'BAT01_20240427_100000.wav': '10:00',
              'BAT01_20240427_110000.wav': '11:00',
              'BAT01_20240427_120000.wav': '12:00',
              'BAT01_20240427_130000.wav': '13:00',
              'BAT01_20240427_140000.wav': '14:00',
              'BAT01_20240427_150000.wav': '15:00'}

# Assuming df2 is your DataFrame and 'file_name' is the column with the filenames
df['file_name'] = df['file_name'].replace(new_labels)

detections = df.groupby('file_name').size()

# Create a bar plot
detections.plot(kind='bar')

# Set the title and labels
plt.title('Number of Detections by Time')
plt.xlabel('Time')
plt.ylabel('Number of Detections')

# Show the plot
plt.show()

## Sources
https://github.com/kahst/BirdNET-Analyzer?tab=readme-ov-file#62-setup-birdnetlib